In [ ]:
import json
import aiohttp
import asyncio
import nest_asyncio
import os
from typing import List

nest_asyncio.apply()

async def fetch_monster_json(session: aiohttp.ClientSession, monster_id: int) -> dict:
    url = f"https://api.hakush.in/zzz/data/en/monster/{monster_id}.json"
    try:
        async with session.get(url) as response:
            if response.status == 200:
                return await response.json()
            return None
    except Exception as e:
        print(f"Error fetching monster {monster_id}: {e}")
        return None

async def process_monsters():
    # Create output directory if it doesn't exist
    os.makedirs('json/monsters', exist_ok=True)
    
    # Load monster IDs
    with open('json/monster.json', 'r', encoding='utf-8') as f:
        monster_data = json.load(f)
    
    monster_ids = [int(monster_id) for monster_id in monster_data.keys()]
    processed_count = 0
    
    async with aiohttp.ClientSession() as session:
        for monster_id in monster_ids:
            result = await fetch_monster_json(session, monster_id)
            if result:
                # Save each monster to its own file
                output_file = f'json/monsters/{monster_id}.json'
                with open(output_file, 'w', encoding='utf-8') as f:
                    json.dump(result, f, indent=4, ensure_ascii=False)
                processed_count += 1
                print(f"Successfully saved monster {monster_id}")
    
    print(f"Processed and saved {processed_count} individual monster files")

# Run the async function using event loop
loop = asyncio.get_event_loop()
loop.run_until_complete(process_monsters())
